# Observando filtro de Canny, e retas encontradas com transformada de Hough (3)

Algoritmo `buscando_moda`:
- executa o algoritmo de canny
  - Avaliar sigma começando em 2.0
  > Incremento de sigma em 0.5
  - algoritmo para após sigma == 5.0
  - calcular `moda` e `contagem_total`
  ```python
  if moda >= 3: 
    # algoritmo para e define essa moda;
  elif moda < moda_anterior: # exceto para a primeira iteração
    # algoritmo para e define a moda dentre a de maior frequência das iterações anteriores
    # (Maior moda, se empate, maior numero de retas)
  
  ```
Se moda começa de 1 e não aumenta, sigma para em 5. E resultado não confiável (avisar).

## Importando bibliotecas

In [ ]:
import numpy as np
import seaborn as sns

from zipfile import ZipFile
from matplotlib import pyplot as plt
from scipy.stats import mode
from skimage.io import imread
from skimage.feature import canny
from skimage.filters import threshold_otsu
from skimage.transform import hough_line, hough_line_peaks

sns.set_style(style='ticks')
plt.gray()

CANNY_SIGMA = 2.0
SIGMA_INCREMENT = 0.5
SIGMA_RANGE = np.arange(CANNY_SIGMA, 5.5, SIGMA_INCREMENT) # [2, 2.5, ..., 5]

LIMITE_DE_NORMALIZACAO = 8

COLOR_NAMES = ['vermelho', 'verde', 'azul']
COLOR_CODES = ['r-', 'g-', 'b-', 'brown-', 'orange-']

In [ ]:
path_list = []
with ZipFile('./toras.zip') as zip_file:
  for path in zip_file.namelist():
    if '__MAC' not in path \
        and '.JPG' in path:
      path_list.append(f'./toras.zip/{path}')

In [ ]:
def buscando_moda(img_mono) -> float:
  '''retorna o melhor sigma'''
  canny_results = []
  moda_freq_anterior = 0
  for canny_sigma in SIGMA_RANGE:
    edges = canny(img_mono, sigma=canny_sigma)
    h, theta, d = hough_line(edges)
    accum, angles, dists = hough_line_peaks(h, theta, d)

    moda = mode(angles, axis=None)
    total_freq = len(angles)
    moda_freq = moda.count[0]
    angle_rad = moda.mode[0]
    angle_deg = -(np.rad2deg(angle_rad) + (90 if angle_rad < 0 else - 90))
    
    resultado ={'sigma':canny_sigma,
                'angulo(deg)':angle_deg,
                'moda_freq':moda_freq,
                'total_freq':total_freq,
                'angles_dists':(angles, dists)}
    if moda_freq >= 3:
      # return resultado
      return canny_sigma
    elif moda_freq < moda_freq_anterior:
      break
    canny_results.append(resultado)
    moda_freq_anterior = moda_freq
  # fora do loop busca o melhor resultado
  best = sorted(canny_results, key=lambda r:(r['angulo(deg)'], r['moda_freq'], r['total_freq']))[0]
  return best['sigma']

In [ ]:
for path in path_list:
  # lendo imagem grayscale
  img_gray = imread(path, as_gray=True)

  # encontrando bordas
  img_otsu = img_gray >= threshold_otsu(img_gray)

  best_sigma = buscando_moda(img_otsu) # float entre 2 e 5 inclusos
  n_plots = 1 + ((best_sigma - 1.5) // 0.5)

  # Plotando imagens
  fig, ax = plt.subplots(nrows=n_plots, ncols=1, figsize=(15, 8))

  # Plotando imagem original
  ax[0].title.set_text(f'{path}')
  ax[0].imshow(img_gray)

  for img_count, canny_sigma in enumerate(range(1, n_plots), start=1):
    edges = canny(img_otsu, sigma=canny_sigma)

    h, theta, d = hough_line(edges)
    accum, angles, dists = hough_line_peaks(h, theta, d)

    moda = mode(angles, axis=None)
    angle = moda.mode[0]
    angle_deg = -(np.rad2deg(angle) + (90 if angle < 0 else - 90))
    count = moda.count[0]
    total = len(angles)
    # Exibindo bordas do canny
    ax[img_count].imshow(edges)
    ax[img_count].set_ylabel(f'sigma={canny_sigma}\n'
                            +f'total de retas: {total}\n'
                            +f'contagem da moda: {count}\n'
                            +f'ângulo: {angle_deg}°',
                            rotation=0, labelpad=60)

    # desenhando retas
    for angulo, distancia in zip(angles, dists):
      if angle == angulo:
        (x0, y0) = distancia * np.array([np.cos(angulo), np.sin(angulo)])
        ax[img_count].axline((x0, y0), slope=np.tan(angulo + np.pi/2), color='r' if best_sigma != canny_sigma else 'b')
        ax[img_count].plot(x0, y0, 'ro') # ponto da reta

  plt.show()
